In [0]:
from google.colab import drive
drive.mount('/content/drive/')


In [0]:
!pwd
%cd "/content/drive/My Drive/c_p/"


In [0]:
import random as rn
import numpy as np
np.random.seed(42)
rn.seed(12345)
!nvidia-smi
%load_ext tensorboard
from tensorflow.python.framework.ops import disable_eager_execution
disable_eager_execution()
import tensorflow as tf
config = tf.compat.v1.ConfigProto() 
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)
import cv2
from tensorflow.keras.utils import multi_gpu_model
from tensorflow.keras.models import Model
from tensorflow.keras import Sequential, optimizers
from tensorflow.keras.layers import Dense, Dropout, Conv2D, Lambda, multiply, ReLU, LeakyReLU, MaxPooling2D,MaxPool2D, GlobalMaxPooling2D, GlobalAveragePooling2D,  SpatialDropout2D,  Flatten, BatchNormalization, concatenate, Input, Activation, Average, AveragePooling2D
from sklearn import svm, metrics
from sklearn.neural_network import MLPClassifier
from helper import Helper
import argparse
from sklearn import tree
import keras
from sklearn.ensemble import AdaBoostClassifier
from keras_model import Keras_model
from sklearn_model import Sklearn_model
from tensorflow.keras.utils import plot_model
from tensorflow.keras import regularizers
import matplotlib.pyplot as plt
import numpy as np
import tensorflow
from collections import Counter
from tqdm.notebook import tqdm
from tensorflow.keras.initializers import Constant
from keras.utils import to_categorical
import datetime
from sklearn.metrics import plot_confusion_matrix
print("Imported")



In [0]:
helper = Helper(mode='all', img_resize=[480, 360])

In [0]:

# # for i in range(100, 200):
# #   if helper.Y_train[i] == 0:
# for i, img in enumerate(helper.X_train_images[:50]):
#     if helper.Y_train[i] == 3:
#       plt.figure(dpi=200)
#       plt.imshow(helper.X_train_images[i])
#       plt.title(helper.Y_train[i])
#       plt.show()


In [0]:

from tensorflow.keras import backend as K
def categorical_focal_loss(gamma=2., alpha=.25):
    """
    Softmax version of focal loss.
           m
      FL = ∑  -alpha * (1 - p_o,c)^gamma * y_o,c * log(p_o,c)
          c=1
      where m = number of classes, c = class and o = observation
    Parameters:
      alpha -- the same as weighing factor in balanced cross entropy
      gamma -- focusing parameter for modulating factor (1-p)
    Default value:
      gamma -- 2.0 as mentioned in the paper
      alpha -- 0.25 as mentioned in the paper
    References:
        Official paper: https://arxiv.org/pdf/1708.02002.pdf
        https://www.tensorflow.org/api_docs/python/tf/keras/backend/categorical_crossentropy
    Usage:
     model.compile(loss=[categorical_focal_loss(alpha=.25, gamma=2)], metrics=["accuracy"], optimizer=adam)
    """
    def categorical_focal_loss_fixed(y_true, y_pred):
        """
        :param y_true: A tensor of the same shape as `y_pred`
        :param y_pred: A tensor resulting from a softmax
        :return: Output tensor.
        """

        # Scale predictions so that the class probas of each sample sum to 1
        y_pred /= K.sum(y_pred, axis=-1, keepdims=True)

        # Clip the prediction value to prevent NaN's and Inf's
        epsilon = K.epsilon()
        y_pred = K.clip(y_pred, epsilon, 1. - epsilon)

        # Calculate Cross Entropy
        cross_entropy = -y_true * K.log(y_pred)

        # Calculate Focal Loss
        loss = alpha * K.pow(1 - y_pred, gamma) * cross_entropy

        # Sum the losses in mini_batch
        return K.sum(loss, axis=1)

    return categorical_focal_loss_fixed


In [0]:
from tensorflow.keras import backend as K
def mse_and_crossentropy():
    def loss(y_true, y_pred):
        loss_cce = tf.keras.losses.categorical_crossentropy(y_true, y_pred)
        loss_mse = tf.keras.losses.MSE(K.cast(K.argmax(y_true, axis=1), dtype=tf.float32), K.cast(K.argmax(y_pred, axis=1), dtype=tf.float32))
        return loss_mse + loss_cce
    return loss

In [0]:
def xcept(output_bias, regularization=None, classes=5):
    if output_bias is not None:
      output_bias = tf.keras.initializers.Constant(output_bias)
    regularizer = None
    dropout_regularizer = None
    if regularization:
      prediction_regularizer = regularizers.l2(regularization[1] * 35)
      prediction_bias_regularizer = regularizers.l1(regularization[0])
      
      regularizer = regularizers.l2(regularization[1])
      bias_regularizer = regularizers.l1(regularization[0])

      activity_regularizer = regularizers.l2(regularization[1])
      prediction_activity_regularizer = regularizers.l2(regularization[1] * 50)

      sparsity = regularizers.l1(regularization[0])

    input_image = Input(shape=(299, 299, 3))
    base_model = tf.keras.applications.Xception(input_tensor=input_image, include_top=False)
    base_model._name = 'm1'
    print("xcpt")
    another_base_model = tf.keras.applications.MobileNetV2(input_tensor=input_image, include_top=False)
    another_base_model._name = 'm2'
    print("mb")

    # for layer in base_model.layers[:int(len(base_model.layers) * 0.75)]:
    #   layer.trainable = False

    # for layer in another_base_model.layers[:int(len(another_base_model.layers) * 0.7)]:
    #   layer.trainable = False

    # for layer in base_model.layers:
    #   for attr in ['kernel_regularizer']:
    #     if hasattr(layer, attr):
    #       # print(attr)
    #       setattr(layer, attr, regularizer)
    
    # for layer in base_model.layers:
    #   for attr in ['bias_regularizer']:
    #     if hasattr(layer, attr):
    #       # print(attr)
    #       setattr(layer, attr, bias_regularizer)

    # for layer in another_base_model.layers:
    #   for attr in ['kernel_regularizer']:
    #     if hasattr(layer, attr):
    #       # print(attr)
    #       setattr(layer, attr, regularizer)
    
    # for layer in another_base_model.layers:
    #   for attr in ['bias_regularizer']:
    #     if hasattr(layer, attr):
    #       # print(attr)
    #       setattr(layer, attr, bias_regularizer)

    for layer in base_model.layers:
      for attr in ['activity_regularizer']:
        if hasattr(layer, attr):
          # print(attr)
          setattr(layer, attr, sparsity)
    
    for layer in another_base_model.layers:
      for attr in ['activity_regularizer']:
        if hasattr(layer, attr):
          # print(attr)
          setattr(layer, attr, sparsity)

    y = another_base_model.output
    y = GlobalAveragePooling2D()(y)
    y = Dropout(0.4)(y)
    y = Dense(512, activation=LeakyReLU(alpha=0.1), activity_regularizer=activity_regularizer)(y)
    y = Dropout(0.3)(y)
    y = Dense(100, activation=LeakyReLU(alpha=0.1), activity_regularizer=activity_regularizer)(y)
    y = Dropout(0.2)(y)

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.4)(x)
    x = Dense(512, activation=LeakyReLU(alpha=0.1), activity_regularizer=activity_regularizer)(x)
    x = Dropout(0.3)(x)
    x = Dense(100, activation=LeakyReLU(alpha=0.1), activity_regularizer=activity_regularizer)(x)
    x = Dropout(0.2)(x)

    # model_1 = Model(input_image, x)
    # model_1.summary()
    # model_2 = Model(base_model.input, y)
    # model_1.summary()
    z = Dense(64, activation=LeakyReLU(alpha=0.1), activity_regularizer=activity_regularizer)(concatenate([x, y]))
    # z = Dropout(0.3)(z)

    predictions = Dense(classes, activation='softmax', activity_regularizer=prediction_activity_regularizer, bias_initializer=output_bias)(z)

    # add your top layer block to your base model
    model = Model(input_image, predictions)
    return model

In [0]:
# initial_bias = np.log([pos/neg])
counts = Counter(helper.Y_train)
total = len(list(helper.Y_train))
bias_factors = []
for i in range(5):
    bias_factors.append(counts[i] / (total - counts[i]))
initial_bias = np.log(bias_factors)
print(initial_bias)

In [0]:
arch = xcept(initial_bias, [0.01, 0.001], classes=5)
# arch.summary()
# from tensorflow.keras.utils import plot_model
# plot_model(arch, to_file='model.png', dpi=300, show_shapes=True)

In [0]:
X = np.array(helper.X_train_images)
Y = np.array(helper.Y_train)
model = Keras_model(helper, arch, use_data_gen=True, is_cnn=True, early_stopping_patience=3, show_summary = False, epochs=34, validation_split=0.15, early_stopping=True, early_stopping_metric='val_loss', k_fold_splits = 4, optimizer = optimizers.Adam(learning_rate=0.00006), batch_size=32, verbose=1)
X_val, Y_val = model.fit(X, Y, [[299, 299], [500, 400, 360, 360, 400], [162, 162, 162, 162, 162]])


In [0]:
plt.figure(dpi=200)
plt.plot(model.history.history['accuracy'])
plt.plot(model.history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.figure(dpi=200)
plt.plot(model.history.history['loss'])
plt.plot(model.history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [0]:
model.model.save('./model_multi_tl_hsv',  overwrite=True, save_format='tf')

In [0]:
np.save('X_train', model.X_train, allow_pickle=True)
np.save('X_val', model.X_val, allow_pickle=True)
np.save('Y_train', model.Y_train, allow_pickle=True)
np.save('Y_val', model.Y_val, allow_pickle=True)

In [0]:
X_train = np.load('X_train.npy', allow_pickle=True)
X_val = np.load('X_val.npy', allow_pickle=True)
Y_train = np.load('Y_train.npy', allow_pickle=True)
Y_val = np.load('Y_val.npy', allow_pickle=True)

In [0]:
loaded_model = tf.keras.models.load_model('./model_multi_tl_hsv', compile=False)
loaded_model.compile(loss='categorical_crossentropy')

In [0]:
# from sklearn.metrics import mean_squared_error, classification_report
# from keras_preprocessing.image import ImageDataGenerator
# data_generator = ImageDataGenerator(dtype='float32', samplewise_center=True, samplewise_std_normalization=True)
# data_generator.fit(X_val)
# print(X_val.shape)
# val_pred = np.argmax(loaded_model.predict(data_generator.flow(X_val, shuffle=False, batch_size=76), steps=10), axis=1)
# print(val_pred)
# print(Y_val.shape)
# accuracy = helper.calc_accuracy(list(np.argmax(Y_val, axis=1)), list(val_pred))
# mse = mean_squared_error(list(np.argmax(Y_val, axis=1)), list(val_pred))
# print("Classification report for - \n{}:\n{}\n".format(loaded_model, metrics.classification_report(list(np.argmax(Y_val, axis=1)), list(val_pred))))
# print(mse)
# print(accuracy)
# print(Counter(val_pred))
# print(accuracy)


In [0]:
# from sklearn.metrics import mean_squared_error, classification_report
# from keras_preprocessing.image import ImageDataGenerator
# X_train_0 = crop_test(helper.X_train_images[0:500], [299, 299])
# X_train_1 = crop_test(helper.X_train_images[500:1000], [299, 299])
# X_train_2 = crop_test(helper.X_train_images[1000:], [299, 299])


# data_generator = ImageDataGenerator(dtype='float32', samplewise_center=True, samplewise_std_normalization=True)
# data_generator.fit(X_train_0)
# softmax_0 = loaded_model.predict(data_generator.flow(X_train_0, shuffle=False, batch_size=100), steps=25)

# data_generator = ImageDataGenerator(dtype='float32', samplewise_center=True, samplewise_std_normalization=True)
# data_generator.fit(X_train_1)
# softmax_1 = loaded_model.predict(data_generator.flow(X_train_1, shuffle=False, batch_size=100), steps=25)
# print(X_train_2.shape)

# data_generator = ImageDataGenerator(dtype='float32', samplewise_center=True, samplewise_std_normalization=True)
# data_generator.fit(X_train_2)
# softmax_test = loaded_model.predict(data_generator.flow(X_train_2, shuffle=False, batch_size=125), steps=11)

y_true_0 = helper.Y_train[0:500]
y_true_1 = helper.Y_train[500:1000]
y_true_test = helper.Y_train[1000:]

softmax_all = np.concatenate([softmax_0, softmax_1, softmax_test])
y_true = np.concatenate([y_true_0, y_true_1, y_true_test])


print(softmax_all.shape)
print(y_true.shape)
results = ensemble_decision(softmax_all)
print(len(results))

accuracy = helper.calc_accuracy(y_true, list(results))
mse = mean_squared_error(y_true, results)
print("Classification report for - \n{}:\n{}\n".format(loaded_model, metrics.classification_report(y_true, results)))
print(mse)
print(accuracy)
print(results)
print(Counter(results))
print(accuracy)
# print(len(softmax_5))
# dont_forget = [softmax_5, y_true]
print(X_train_0.shape)

In [0]:
from sklearn.ensemble import RandomForestRegressor
def decision_clf(softmax, y_true):
  regr = RandomForestRegressor(random_state=0)
  regr.fit(softmax, y_true)

  print(regr.feature_importances_)  
  return regr

def check_decision_clf(all_preds, index, y_true):
    global k
    k = 0
    softmax = []
    ind = []
    for i in range(index, len(all_preds), 5):
        softmax.append(all_preds[i:i+5].flatten())
        ind.append(i)
    print(ind)
    regr = decision_clf(softmax, y_true)
    pred = np.round(regr.predict(softmax)).astype('int')
    print(pred)
    acc = helper.calc_accuracy(y_true, list(pred))
    print(acc)
    return regr

c_0 = check_decision_clf(softmax_all, 0, y_true)

def ensemble_decision(softmax_5):

    softmax = []
    for i in range(0, len(softmax_5), 5):
          softmax.append(softmax_5[i:i+5].flatten())
    pred = np.round(c_0.predict(softmax)).astype('int')
    return pred

# c_1 = check_classifer(softmax_5, 1, y_true)
# # print(c_1)
# c_2 = check_classifer(softmax_5, 2, y_true)
# # print(c_2)
# c_3 = check_classifer(softmax_5, 3, y_true)
# # print(c_3)
# c_4 = check_classifer(softmax_5, 4, y_true)
# # print(c_4)



In [0]:
def calc_accuracy(thresholds, softmax, y_true, k):
    softmax = np.array(softmax)
    total = np.array(y_true).shape[0]
    correct = 0
    if total != softmax.shape[0]:
      print("unequal y ypred")
    for i, y in enumerate(softmax):
        # print(len(y))
        factor = 0
        volume = 0
        most_confident = np.argmax(y)
        for c in range(5):
            volume += (y[c] * thresholds[most_confident * 5 + c]) * c
            factor = (y[c] * thresholds[most_confident * 5 + c])
        if (int(round(volume/factor)) == y_true[i]): correct += 1
    # print(correct)
    return -1 * (correct)

bounds = [(-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1), (-1, 1)]


In [0]:

from scipy.optimize import differential_evolution
def check_classifer(all_preds, index, y_true):
    global k
    k = 0
    softmax = []
    ind = []
    for i in range(index, len(all_preds), 5):
        softmax.append(all_preds[i])
        ind.append(i)
    print(ind)
    return differential_evolution(calc_accuracy, bounds, args=(softmax, y_true, k))

In [0]:
c_0 = check_classifer(softmax_5, 0, y_true)
print(c_0)
# c_1 = check_classifer(softmax_5, 1, y_true)
# print(c_1)
# c_2 = check_classifer(softmax_5, 2, y_true)
# print(c_2)
# c_3 = check_classifer(softmax_5, 3, y_true)
# print(c_3)
# c_4 = check_classifer(softmax_5, 4, y_true)
# print(c_4)

In [0]:
# from sklearn.metrics import mean_squared_error
# from keras_preprocessing.image import ImageDataGenerator
# data_generator = ImageDataGenerator(dtype='float32', samplewise_center=True, samplewise_std_normalization=True)
# data_generator.fit(model.X_train)
print(len(model.X_train))
# # softmax_5 = loaded_model.predict(data_generator.flow(X_train, shuffle=False, batch_size=75), steps=len(X_train/75))
model_train_pred = model.model.predict(data_generator.flow(model.X_train, shuffle=False, batch_size=100), steps=len(model.X_train/100) + 1)

# model.print_classification_report(np.argmax(model.Y_train, axis=1), model_train_pred)
# accuracy = helper.calc_accuracy(np.argmax(model.Y_train, axis=1), list(model_train_pred))
# mse = mean_squared_error(np.argmax(model.Y_train, axis=1), model_train_pred)
# print(mse)
# print(model_train_pred)
# print(accuracy)
# print(Counter(model_train_pred))


In [0]:
all_th = [c_0.x, c_1.x, c_2.x, c_3.x, c_4.x]
def classify(thresholds, softmax):
    softmax = np.array(softmax)
    votes = []
    for i, y in enumerate(softmax):
        factor = 0
        volume = 0
        for c in range(5):
            volume += (y[c] - thresholds[c]) * i
            factor = (y[c] - thresholds[c])
        votes.append(int(round(volume/factor)) == y_true[i])
            
    # print(correct)
    return votes

def predict_ensemble(all_softmax_5):
    results = []
    votes = []
    for index in range(5):
      softmax = []
      for i in range(index, len(all_softmax_5), 5):
          softmax.append(all_softmax_5[i])
      votes_i = classify(all_th[index], softmax)
      # print(votes_i)
      votes.append(votes_i)
    print(votes)

    for i in range(0, int(len(softmax))):
        print(i)
        sample_votes = []
        for c in range(5):
          sample_votes.append(votes[c][i])
        vote_count = np.bincount(sample_votes)
        choice = np.argmax(vote_count)
        # print(choice)
        results.append(choice)
    return results


In [0]:
# from keras_preprocessing.image import ImageDataGenerator
# data_generator = ImageDataGenerator(dtype='float32', samplewise_center=True, samplewise_std_normalization=True)
# pred_val = model.model.predict(data_generator.flow(model.X_val, model.Y_val, shuffle=False, batch_size=100), steps=10)
model.print_classification_report(np.argmax(model.Y_val, axis=1), np.argmax(pred_val, axis=1))
accuracy = helper.calc_accuracy(np.argmax(model.Y_val, axis=1), np.argmax(pred_val, axis=1))
mse = mean_squared_error(np.argmax(model.Y_val, axis=1), np.argmax(pred_val, axis=1))
print(mse)
print(accuracy)
print(Counter(np.argmax(pred_val, axis=1)))


In [0]:
from sklearn.metrics import mean_squared_error, classification_report
X_train = crop_test(helper.X_train_images, dim=[299, 299])
pred = loaded_model.predict_classes(X_train[:100])
print(helper.Y_train)
results = predict_cropped_test_max(pred)
accuracy = helper.calc_accuracy(helper.Y_train[:100], list(results))
mse = mean_squared_error(helper.Y_train[:100], results)
print("Classification report for - \n{}:\n{}\n".format(model, metrics.classification_report(helper.Y_train[:100], results)))
print(mse)
print(accuracy)
print(results[:100])
print(Counter(results))

In [0]:

from keras_preprocessing.image import ImageDataGenerator
data_generator = ImageDataGenerator(dtype='float32', samplewise_center=True, samplewise_std_normalization=True)

# X_test = crop_test(helper.X_test_images, dim=[299, 299])
# data_generator.fit(X_test)
# pred = model.model.predict(data_generator.flow(X_test, shuffle=False, batch_size=100), steps=10)

results = predict_ensemble(pred)
print(results)
print(Counter(results))

# pred_1, pred_2, pred_3, pred_4  = predict_cropped_test(pred)
# print(pred_1)
# print(Counter(pred_1))
# print(pred_2)
# print(Counter(pred_2))
# print(pred_3)
# print(Counter(pred_3))
# print(pred_4)
# print(Counter(pred_4))
# voted_predictions = helper.one_hot_transform(helper.one_hot, results)
# np.savetxt("predict.csv", voted_predictions, delimiter=",", fmt='%i')
# print(voted_predictions)

In [0]:
import cv2
for i, img in enumerate(helper.X_test_images):
  if results[i] == 4:
    plt.figure(dpi=200)
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_HSV2RGB))
    plt.title(str(results[i]) + "   " + str(i))
    plt.show()

In [0]:
import cv2
def crop_test(X, dim):
    data = []
    original_dim_x = X[0].shape[1]
    original_dim_y = X[0].shape[0]
    mid_start_x = int((original_dim_x - dim[0])/2)
    mid_start_y = int((original_dim_y - dim[1]) / 2)
    for x in X:
        data.append(x[:dim[1], :dim[0]])
        data.append(x[-dim[1]:, -dim[0]:])
        data.append(x[-dim[1]:, :dim[0]])
        data.append(x[:dim[1], -dim[0]:])
        data.append(x[mid_start_y:mid_start_y+dim[1], mid_start_x:mid_start_x+dim[0]])
    del X
    return np.array(data)



def predict_ensemble_center(pred, images=None):
    results = []
    for i in range(0, len(pred), 5):
        print(pred[i:i+5])
        softmax = pred[i+4]
        print(softmax)
        choice = np.argmax(softmax)
        results.append(choice)
        print(choice)
    return results

def predict_ensemble_rest(pred, images=None):
    results = []
    for i in range(0, len(pred), 5):
        softmax = pred[i:i+3]
        print(softmax)
        total_prob = np.sum(softmax, axis=0)
        total_prob /= np.sum(total_prob)
        choice = np.argmax(total_prob)
        results.append(choice)
        print(choice)
    return results


def predict_ensemble(pred, images=None):
    results = []
    for i in range(0, len(pred), 5):
        softmax = pred[i:i+5]
        total_prob = np.sum(softmax, axis=0)
        total_prob /= np.sum(total_prob)
        choice = np.argmax(total_prob)
        results.append(choice)
    return results

def predict_ensemble_top(pred, images=None):
    results = []
    for i in range(0, len(pred), 5):
        softmax = pred[i:i+5]
        print(softmax)
        top = np.max(softmax, axis=0)
        print(top)
        choice = np.argmax(top)
        results.append(choice)
        print(choice)
    return results

def predict_ensemble_vote(pred, images=None):
    results = []
    for i in range(0, len(pred), 5):
        softmax = pred[i:i+5]
        print(softmax)
        votes = np.argmax(softmax, axis=1)
        print(votes)
        counts = np.bincount(votes)
        print(counts)
        choice = np.argmax(counts)
        results.append(choice)
        print(choice)
    return results

def predict_ensemble_top_3(pred, images=None):
    results = []
    for i in range(0, len(pred), 5):
        softmax = np.array(pred[i:i+5])
        print(softmax)
        top_softmax = np.sort(softmax, axis=0)[2:]
        print(top_softmax)
        total_prob = np.sum(softmax, axis=0)
        total_prob /= np.sum(total_prob)
        choice = np.argmax(total_prob)
        results.append(choice)
        print(choice)
    return results

# def predict_ensemble_top_3(pred, images=None):
#     results = []
#     for i in range(0, len(pred), 5):
#         softmax = np.array(pred[i:i+5]).T
#         top_softmax = np.sort(softmax)[2:].T
#         print(top_softmax)
#         total_prob = np.sum(softmax, axis=0)
#         total_prob /= np.sum(total_prob)
#         choice = np.argmax(total_prob)
#         results.append(choice)
#         print(choice)
#     return results

def predict_ensemble_top_volume(pred, images=None):
    results = []
    for i in range(0, len(pred), 5):
        softmax = np.array(pred[i:i+5])
        print(softmax)
        top_softmax = np.sort(softmax, axis=0)[3:]
        print(top_softmax)
        factor = 0
        volume = 0
        for i in range(5):
          for j in range(2):
            volume += top_softmax[j][i] * i
            factor += top_softmax[j][i]
        choice = int(round(volume/factor))
        results.append(choice)
    return results


In [0]:
# model.model.save('./model_tl',  overwrite=True, save_format='tf', include_optimizer=False)

In [0]:
# model_01_234.model.save('./binary_models/model_01_234',  overwrite=True, save_format='tf')
model = tf.keras.models.load_model('./binary_models/model_1_3')

In [0]:
# X, Y = helper.aug_by_crop(helper.X_train_images, helper.Y_train, [400, 400], [500, 400, 400, 400, 400])

In [0]:
model_1_3.model = model

In [0]:
def decide(pred, negative_classes, final_pred):
  if len(negative_classes) == 1:
    negative_class = negative_classes[0]
    for i, p in enumerate(pred):
      if p == 0 and type(final_pred[i]).__name__ != 'list' and final_pred[i] == -1:
        final_pred[i] = negative_class
      elif p == 0 and type(final_pred[i]).__name__ == 'list' and negative_class in final_pred[i]:
        final_pred[i] = negative_class
  else:
    for i, p in enumerate(pred):
      if p == 0 and type(final_pred[i]).__name__ != 'list' and final_pred[i] == -1:
        final_pred[i] = negative_classes
      elif p == 0 and type(final_pred[i]).__name__ == 'list' and all([1 if e in final_pred[i] else 0 for e in negative_classes]):
        final_pred[i] = negative_classes
  return final_pred

def reset_pred(classes, final_pred):
  for i, p in enumerate(final_pred):
    if p in classes:
      final_pred[i] = -1
  return final_pred

In [0]:
model_01_234 = tf.keras.models.load_model('./binary_models/model_01_234')
print("loaded")
model_0_1 = tf.keras.models.load_model('./binary_models/model_0_1')
print("loaded")
model_4_23 = tf.keras.models.load_model('./binary_models/model_4_23')
print("loaded")
model_3_4 = tf.keras.models.load_model('./binary_models/model_3_4')
print("loaded")
model_1_23 = tf.keras.models.load_model('./binary_models/model_1_23')
print("loaded")
model_2_3 = tf.keras.models.load_model('./binary_models/model_2_3')
print("loaded")
model_1_2 = tf.keras.models.load_model('./binary_models/model_1_2')
print("loaded")
model_1_3 = tf.keras.models.load_model('./binary_models/model_1_3')
print("loaded")


In [0]:
X_test = np.array(helper.X_test_images)
pred_01_234 = np.argmax(model_01_234.predict(X_test), axis=1)
pred_0_1 = np.argmax(model_0_1.predict(X_test), axis=1)
pred_4_23 = np.argmax(model_4_23.predict(X_test), axis=1)
pred_3_4 = np.argmax(model_3_4.predict(X_test), axis=1)
pred_1_23 = np.argmax(model_1_23.predict(X_test), axis=1)
pred_2_3 = np.argmax(model_2_3.predict(X_test), axis=1)
pred_1_2 = np.argmax(model_1_2.predict(X_test), axis=1)
pred_1_3 = np.argmax(model_1_3.predict(X_test), axis=1)

In [0]:



final_pred = [-1] * X_test.shape[0]

final_pred = decide(pred_01_234, [0, 1], final_pred)
final_pred = decide(np.logical_not(pred_01_234), [2, 3, 4], final_pred)

final_pred = decide(pred_0_1, [0], final_pred)
final_pred = decide(np.logical_not(pred_0_1), [1], final_pred)


final_pred = decide(pred_4_23, [4], final_pred)
final_pred = decide(np.logical_not(pred_4_23), [2], final_pred)

final_pred = reset_pred([1, 2, 3], final_pred)

final_pred = decide(pred_1_23, [1], final_pred)
final_pred = decide(np.logical_not(pred_1_23), [3], final_pred)


final_pred = decide(pred_2_3, [2], final_pred)
final_pred = decide(np.logical_not(pred_2_3), [3], final_pred)

# final_pred = decide(np.logical_not(pred_1_3), [3], final_pred)
# final_pred = decide(np.logical_not(pred_1_2), [2], final_pred)
# final_pred = decide(pred_1_2, [1], final_pred)
# final_pred = decide(pred_1_3, [1], final_pred)



final_pred = reset_pred([3, 4], final_pred)
final_pred = decide(pred_3_4, [3], final_pred)
final_pred = decide(np.logical_not(pred_3_4), [4], final_pred)


final_pred = reset_pred([1, 3], final_pred)
final_pred = decide(pred_1_3, [1], final_pred)
final_pred = decide(np.logical_not(pred_1_3), [1], final_pred)





In [0]:
print(final_pred)
print(Counter(final_pred))
voted_predictions = helper.one_hot_transform(helper.one_hot, final_pred)
np.savetxt("predict.csv", voted_predictions, delimiter=",", fmt='%i')
print(voted_predictions)

In [0]:
import cv2
for i, img in enumerate(X_test):
  if final_pred[i] == 3:
    plt.figure(dpi=200)
    plt.imshow(cv2.cvtColor(X_test[i], cv2.COLOR_BGR2RGB))
    plt.title(final_pred[i])
    plt.show()


In [0]:
X_0_selected_indices = np.random.choice(X_0.shape[0], size=50, replace=False) 
X_0_selected = X_0[X_0_selected_indices]